# Linpackフロントエンドの作成

## 前提

このnotebookでは、OpenHPC環境においてLinpackベンチマークの実行環境を整備し、Open OnDemandをフロントエンドとしてLinpackを実行する手順を示します。このnotebookの実行にあたっては、以下を前提とします。

* 030-ジョブ実行環境の設定のnotebookによりOpen OnDemandの設定を完了し、Open OnDemandからジョブ実行が可能になっていること。

## 準備

### group_varsの読み込み

group_varsの読み込みに先立ち、ユニットグループ名をチェックします。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

`ugroup_name`にユニットグループ名を設定します。

In [ ]:
ugroup_name = 

group_varsを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

###  ansibleの動作確認

ansibleでSlurmクラスタにアクセスできることを確認します。

In [ ]:
!ansible -m ping {ugroup_name}

ansibleで指定するノード集合を設定します。

In [ ]:
target_master = f'{ugroup_name}_master'
target_compute = f'{ugroup_name}_compute'

## Slurmの設定変更

ベンチマークにあたり、計算ノードのCPU資源を全て使用できるよう、slurm.confにノードあたりのCPU数の記述を追加します。既に記述済みの場合や、ノードあたり1MPIプロセスの実行で十分な場合にはこの節の設定は不要です。
ここでは、SlurmクラスタはVM上に構築されており、ハードウェア上のCPUトポロジがVM上のCPUトポロジに反映されているとは限らないため、CPUトポロジは考慮に入れず、単にCPU数のみを設定することとします。

まず、計算ノードのCPUコア数を取得します。

In [ ]:
out=!ansible '{target_compute}[0]' -a 'hwloc-calc --number-of core machine:0'
print(out[1])

OCS templatesの`OpenHPC-v2/notebooks/031-設定ファイルの編集-slurm_conf`に従った方法で`slurm.conf`を編集します。

以下のセルを実行すると、マスターノードから`slurm.conf`を取得し、`slurm.conf`の編集のためのリンクが表示されます。NodeNameの行が以下のように設定されている場合:  
```
NodeName=c[0-3] State=UNKNOWN
```
ノードあたりのCPU数が3であれば、以下のように編集します:
```
NodeName=c[0-3] CPUs=3 State=UNKNOWN
```

In [ ]:
slurm_conf_path='/etc/slurm/slurm.conf'

%run scripts/edit_conf.py
fetch_conf(slurm_conf_path, ugroup_name, 'master')

以下のセルを実行して、変更した内容を確認します。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(slurm_conf_path, ugroup_name, 'master')

編集内容が正しいことを確認したら、以下のセルを実行してファイルをマスターノードにアップロードします。

In [ ]:
%run scripts/edit_conf.py
upload_conf(slurm_conf_path, ugroup_name)

Slurmの各デーモンに設定のリロードを指示します。

In [ ]:
!ansible {target_master} -b -a 'systemctl restart slurmctld'
!ansible {target_compute} -b -a 'systemctl reload slurmd'

CPU数の設定が反映されていることを確認します。

In [ ]:
!ansible {target_master} -a 'sinfo -o "%P %.5a %.10l %.6D %.3c %.6t %N"'

## HPL (High-Performance Linpack)のビルド

ここでは、OCS templatesの`OpenHPC-v2/notebooks/121-Linpackベンチマーク-HPL`に従った方法で、HPLをビルドします。

Open OnDemandから使用する前提であるため、Open OnDemandで使用するユーザでHPLをビルドします。以下の`ooduser`にそのユーザ名を設定します(セットアップ時に変更していなければユーザ名は`ooduser`です)。

In [ ]:
ooduser = 

HPL環境を構築するディレクトリを指定します。

In [ ]:
hpl_dir = f'/home/{ooduser}'

以下のセルにより、上記のディレクトリが存在していなければ作成します。

In [ ]:
!ansible {target_master} -m file -a \
    'path={hpl_dir} owner={ooduser} group={ooduser} state=directory'

HPLのソースアーカイブをダウンロードします。

In [ ]:
hpl_version = '2.3'
hpl_url = f'https://www.netlib.org/benchmark/hpl/hpl-{hpl_version}.tar.gz'
ansible_opt = f'-b --become-user={ooduser}'

!ansible {target_master} -m get_url {ansible_opt} -a \
    'url={hpl_url} dest={hpl_dir}'

ソースアーカイブを展開します。展開してできたディレクトリを後で参照しやすいようシンボリックリンクを作成します。

In [ ]:
!ansible {target_master} {ansible_opt} -a \
    'chdir={hpl_dir} tar xf hpl-{hpl_version}.tar.gz'
!ansible {target_master} {ansible_opt} -a \
    'chdir={hpl_dir} ln -sf hpl-{hpl_version} hpl'

HPLをビルドするときのarch名を指定します。ここで指定した名前を使って後でMake.{arch}ファイルを作成します。

In [ ]:
hpl_arch = 'Linux_OPENBLAS'

`Make.{arch}`ファイルを作成します。必要に応じて`HPL_OPTS`に追加・変更します。

In [ ]:
!ansible {target_master} {ansible_opt} -m copy -a \
    'remote_src=true src={hpl_dir}/hpl/setup/Make.UNKNOWN.in \
    dest={hpl_dir}/hpl/Make.{hpl_arch}'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@ARCH@ replace={hpl_arch}'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@LADIR@ replace=$(OPENBLAS_DIR)'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@LAINC@ replace=-I$(OPENBLAS_INC)'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@LALIB@ replace="-Wl,-rpath,$(OPENBLAS_LIB) -L$(OPENBLAS_LIB) -lopenblas"'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@SHELL@ replace=/bin/sh'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@CD@ replace=cd'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@CP@ replace=cp'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@LN_S@ replace="ln -s"'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@MKDIR@ replace=mkdir'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@RM@ replace="/bin/rm -f"'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@TOUCH@ replace=touch'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@CC@ replace=mpicc'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@CCNOOPT@ replace=""'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@CCFLAGS@ replace="-fomit-frame-pointer -O3 -funroll-loops -W -Wall"'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@LINKER@ replace=mpif77'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@LINKFLAGS@ replace=$(CCFLAGS)'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@ARCHIVER@ replace=ar'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@ARFLAGS@ replace=r'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@RANLIB@ replace=echo'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@MPDIR@ replace=""'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@MPINC@ replace=""'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@MPLIB@ replace=""'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp=@F2CDEFS@ replace=""'
!ansible {target_master} {ansible_opt} -m replace -a \
    'dest={hpl_dir}/hpl/Make.{hpl_arch} \
    regexp="^HPL_OPTS.*" replace="HPL_OPTS = -DHPL_CALL_CBLAS"'

作成した`Make.{arch}`の内容を確認します。

In [ ]:
!ansible {target_master} {ansible_opt} -a \
    'cat {hpl_dir}/hpl/Make.{hpl_arch}'

HPLをビルドします。

In [ ]:
!ansible {target_master} {ansible_opt} -m shell -a \
    'chdir={hpl_dir}/hpl \
    source ~/.bashrc && module load openblas && make arch={hpl_arch}'

`xhpl`コマンドがビルドできたことを確認します。

In [ ]:
!ansible {target_master} {ansible_opt} -a \
    'ls -l {hpl_dir}/hpl/bin/{hpl_arch}/xhpl'

## Open OnDemandフロントエンドの配置

HPLをSlurmのジョブとして実行するための、Open OnDemandのフロントエンドを配置します。ここでは、Open OnDemandからHPLの実行に必要な代表的なパラメータを設定可能とし、設定された値から`HPL.dat`を生成した上でHPLのジョブを実行するフロントエンドを配置します。

設定可能とするパラメータは以下とします。
* ノード数
* MPIプロセス数
* HPL.datのN値 (問題のサイズ)
* HPL.datのNB値 (ブロックサイズ)
* HPL.datのP値
* HPL.datのQ値

まず、HPL用のフロントエンドを配置するディレクトリを作成します。

In [ ]:
!ansible {target_master} -b -m file -a \
    'path=/var/www/ood/apps/sys/hpl mode=0755 state=directory'

HPLのフロントエンドに必要なファイルを、作成したディレクトリに配置します。

In [ ]:
!ansible {target_master} -b -m copy -a \
    'src=hplfiles/ dest=/var/www/ood/apps/sys/hpl mode=preserve'